## Clean CSV code like a basic clean

In [55]:
import pandas as pd

# Load the CSV into a DataFrame
file_path = 'dbird_data_Georgia_Tech_final(1).csv' 
dfoG = pd.read_csv(file_path)

# Check for duplicate rows
duplicate_rows = dfoG[dfoG.duplicated()]
print(f"Number of duplicate rows: {len(duplicate_rows)}")

# Drop the columns 'notes' and 'image'
dfoG = dfoG.drop(columns=['notes', 'image', 'created_at_utc','updated_at_utc', 'location_accuracy_meters', 'mobile_device_gps_source'])


# Save the updated DataFrame back to a CSV if needed
dfoG.to_csv('updated_dbird.csv', index=False)


#so I know when clena is done
print("Sucessfully cleaned")

Number of duplicate rows: 0
Sucessfully cleaned


## Use the new cleaned CSV data to calculate everything

In [ ]:
import pandas as pd

file_path = 'updated_dbird.csv'
df = pd.read_csv('updated_dbird.csv')

# Define latitude and longitude bounds
top_latitude = 33.77592
bottom_latitude = 33.77495
left_longitude = -84.40163
right_longitude = -84.40106



In [ ]:
# Convert 'observed_at_utc' column to datetime
df['observed_at_utc'] = pd.to_datetime(df['observed_at_utc'], format='mixed', errors='coerce')

# Format the datetime to only include the date (YYYY-MM-DD)
df['observed_at_utc'] = df['observed_at_utc'].dt.date

In [ ]:
# Filter rows within the latitude and longitude range
df_2024_within_bounds = df[
    (df['latitude'] <= top_latitude) &
    (df['latitude'] >= bottom_latitude) &
    (df['longitude'] >= left_longitude) &
    (df['longitude'] <= right_longitude)
]
# Rows in 2024 but outside bounds
df_2024_outside_bounds = df.drop(df_2024_within_bounds.index)

In [ ]:
df_2024_within_bounds.to_csv('dbird_2024_within_bounds.csv', index=False)
df_2024_outside_bounds.to_csv('dbird_2024_outside_bounds.csv', index=False)

print("Files saved successfully!")

Files saved successfully!


In [ ]:
df_2024_outside_bounds.to_csv('dbird_2024_outside_bounds.csv', index=False)

print("Files saved successfully!")

## Calcualting the Mean and std of dbird_2024_outside_bounds (df1)

All the data saved from this run is in the yearly_data where it's split off by year

In [ ]:
import pandas as pd
import os



# File path to the input dataset
file_path = "dbird_2024_outside_bounds.csv"

# Load the dataset
df1 = pd.read_csv(file_path)

# Ensure the 'observed_at_utc' column is in datetime format
df1['observed_at_utc'] = pd.to_datetime(df1['observed_at_utc'], errors='coerce')

# Extract the year from the 'observed_at_utc' column
df1['year'] = df1['observed_at_utc'].dt.year

# Folder to save the yearly CSV files
output_folder = "yearly_data"

# Create the folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Split the data into yearly CSV files (2017-2024)
for year in range(2017, 2025):
    year_df = df1[df1['year'] == year]  # Filter for the current year
    output_file = os.path.join(output_folder, f'dbird_data_{year}.csv')  # Create file path in the folder
    year_df.to_csv(output_file, index=False)  # Save to a CSV file
print("Success: Saved data for 2017-2024.")

# Initialize a list to store the sizes of each year's dataset
sizes = []

# Calculate the size (number of rows) for each CSV file
for year in range(2017, 2025):
    filepath = os.path.join(output_folder, f'dbird_data_{year}.csv')  # Update path to include folder
    try:
        tmeancalc = pd.read_csv(filepath)
        sizes.append(len(tmeancalc))  # Add the number of rows to the sizes list
    except FileNotFoundError:
        print(f"File {filepath} not found. Skipping...")

# Calculate the mean of the sizes
mean_size = sum(sizes) / len(sizes) if sizes else 0

# Calculate the standard deviation of the sizes
std_deviation = pd.Series(sizes).std()

# Output the results
print(f"Number of files processed: {len(sizes)}")
print("")
print(f"Mean size of datasets: {mean_size}")
print(f"Standard deviation of dataset sizes: {std_deviation}")
print("")
print(f"bird data within bounds mean: {len(pd.read_csv('dbird_2024_within_bounds.csv'))}")
print("bird data within bounds Standard deviation: 0")

Success: Saved data for 2017-2024.
Number of files processed: 8

Mean size of datasets: 61.25
Standard deviation of dataset sizes: 78.75413821326651

bird data within bounds mean: 11
bird data within bounds Standard deviation: 0


## Calculating the bird Species (df2)

In [5]:
import pandas as pd

file_path = "updated_dbird.csv"

df2 = pd.read_csv(file_path)

cleaned_df = df2[(df2["common_name"]!='unknown' )]

bird_counts = cleaned_df['common_name'].value_counts()

print(bird_counts)

common_name
Ovenbird                   44
Tennessee Warbler          28
Gray Catbird               22
American Robin             21
Swainson's Thrush          19
                           ..
White-breasted Nuthatch     1
Dark-eyed Junco             1
Carolina Chickadee          1
Red-shouldered Hawk         1
Hooded Warbler              1
Name: count, Length: 70, dtype: int64
